#Этап 3. GPT. Создание прототипа НС - версия 3.0

отчет: https://docs.google.com/document/d/1jpH80B3WkXyzlZ3JMY4pT6ckCg8lnqvm5egz7bHx-pA/edit?usp=sharing

## Установка и настройка НС

In [ ]:
#@title Установка библиотек
!pip install -q requests beautifulsoup4 ffmpeg requests-html nest_asyncio pydub SpeechRecognition lxml_html_clean requests-html
#!pip install --upgrade requests-html google-genai
!pip -q install faster-whisper==1.0.3 ctranslate2==4.4.0
!pip -q install yt-dlp -U
!pip -q install pyaspeller
!pip install --upgrade langchain_openai faiss-cpu openai tiktoken langchain_community langchain

In [ ]:
#@title Импорт модулей и библиотек
import nest_asyncio
import re
import requests
from pydub import AudioSegment
import yt_dlp
import pandas as pd
import nest_asyncio
from requests_html import AsyncHTMLSession
import re
import yt_dlp
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from faster_whisper import WhisperModel
from pyaspeller import YandexSpeller
from google.colab import drive
import shutil

import numpy as np
import io
import tempfile
import tiktoken
from scipy.spatial.distance import cdist
import json
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, CharacterTextSplitter
import openai
from openai import OpenAI
from langchain.docstore.document import Document as Doc
import tiktoken
import matplotlib.pyplot as plt
from google.colab import userdata

# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Получение ключа GPT
openai_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Конфигурация НС

l = '**********Системные настройки*********' #@param ['**********Системные настройки*********']
# Путь к файлу с сылками на лекции
video_list_patch = '/content/drive/MyDrive/MGPU/Лекции.xlsx' # @param {type:"string", description: "Путь к файлу с сылками на лекции"}
# Путь к папке на Google Диске
folder_path = "/content/drive/My Drive/MGPU/audio/" #@param {type:"string"}
# Укажите путь к файлу индекса
index_path = "/content/drive/MyDrive/MGPU/audio/faiss_index.bin" #@param {type:"string"}
# Размер чанков в токенах для формироавния базы знаний
pcs_token = 500 # @param {type: "integer"}

l = '*********Конфигурация GPT*********' #@param ['*********Конфигурация GPT*********']
# Конфигурация GPT
# модель GPT
model_gpt = 'gpt-4o-mini' #@param ['gpt-4o-mini', 'gpt-4o', 'gpt-4']
# Температура
temperature = 0.1  # @param {type: "slider", min: 0.0, max: 1, step: 0.1}
# Промт для формирования экзаменационного вопроса
system_answer = 'Вы лучший преподаватель Психолого-педагогического института. Ответьте развернуто на вопрос, на основе отрывков из лекций. При формировании ответа на лекции ссылаться не надо.' # @param {type:"string"}
# Промты для формирования пересказа
# Промт для формирования пересказа части
system_part = 'Перескажи текст, сохраняя смысл, но меняя формулировки' # @param {type:"string"}
# Промт для формирования пересказа всей лекции
system_all = 'Объедини текст в единый пересказ, устрани повторы и улучши связность' # @param {type:"string"}

# Конфигурация Faster Whisper
l = '*********Конфигурация Faster Whisper*********' #@param ['*********Конфигурация Faster Whisper*********']
model_whisper = "small" #@param ["small", "medium", "large-v3"]
# Параметр с условием выбора GPU
use_gpu = False  # @param {type: "boolean"}


In [ ]:
#@title Загрузка модели Faster Whisper
if use_gpu:
    device = "cuda"
else:
    device = "cpu"
print(f"Используется устройство: {device}")

model = WhisperModel(model_whisper, device = device, compute_type="int8")

Используется устройство: cpu


vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

## Функции скачивания аудио и транскрибации

In [ ]:
#@title Парсинг Dzen по ссылке video/watch и получение ссылки dzen.ru/embed/

async def get_dzen_embed (url):
    # Применяем nest_asyncio
    nest_asyncio.apply()

    # Создаем асинхронную сессию
    session = AsyncHTMLSession()

    # Заголовки для имитации запроса от браузера
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    async def fetch_video():
        try:
            # Открываем страницу с заголовками
            response = await session.get(url, headers=headers)

            # Ждем, чтобы JavaScript выполнился (если это необходимо)
            await response.html.arender(sleep=10, timeout=60)  # Увеличьте время ожидания

            # Получаем HTML-код страницы
            html_content = response.html.html

            # Используем регулярное выражение для поиска ссылки dzen.ru/embed/
            embed_link = re.search(r'https://dzen\.ru/embed/[^\s"]+', html_content)

            if embed_link:
                return embed_link.group(0)  # Возвращаем найденную ссылку
            else:
                print("Ссылка не найдена.")
                return None  # Возвращаем None, если ссылка не найдена

        except Exception as e:
            print(f'Произошла ошибка: {e}')
            return None  # Возвращаем None в случае ошибки

    # Запускаем асинхронную функцию
    embed_url = await fetch_video()
    print('Ссылка dzen', embed_url)  # Выводим найденную ссылку
    return embed_url

In [ ]:
#@title Парсинг Dzen по ссылке video/watch и получение ссылки s3.dzeninfra.ru

async def get_url_dzen(url):
    embed_url = await get_dzen_embed(url)

    if embed_url is None:
        print("Не удалось получить embed URL. Пропускаем...")
        return None, "lecture"  # Возвращаем None и название по умолчанию

    print(embed_url)
    # Применяем nest_asyncio
    nest_asyncio.apply()

    # Создаем асинхронную сессию
    session2 = AsyncHTMLSession()

    # Заголовки для имитации запроса от браузера
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    async def fetch_video2(url=embed_url):
        print(url)
        try:
            # Открываем страницу с заголовками
            response = await session2.get(url, headers=headers)

            # Ждем, чтобы JavaScript выполнился (если это необходимо)
            await response.html.arender(sleep=10, timeout=60)  # Увеличьте время ожидания

            # Получаем HTML-код страницы
            html_content = response.html.html

            # Используем регулярное выражение для поиска ссылки s3.dzeninfra.ru
            audio_link = re.search(r'https://s3\.dzeninfra\.ru/[^\s"]+/audio/[^\s"]+', html_content)

            # Используем регулярное выражение для поиска мета-тега с названием лекции
            title_match = re.search(r'<meta name="vh-title" content="([^"]+)"', html_content)
            lecture_title = title_match.group(1) if title_match else "lecture"

            if audio_link:
                return audio_link.group(0), lecture_title  # Возвращаем найденную ссылку и название лекции
            else:
                print("Ссылка не найдена.")
                return None, lecture_title  # Возвращаем None, если ссылка не найдена

        except Exception as e:
            print(f'Произошла ошибка: {e}')
            return None, "lecture"  # Возвращаем None в случае ошибки

    # Запускаем асинхронную функцию
    audio_url, lecture_title = await fetch_video2()
    print(audio_url)  # Выводим найденную ссылку
    print(lecture_title)  # Выводим название лекции
    return audio_url, lecture_title

In [ ]:
#@title Скачивание аудиофайла c Dzen

def get_dzen_audio (lecture_title, audio_url):
    # Форматируем имя файла для сохранения
    file_name = f"{lecture_title}.mp4".replace('/', '_').replace('\\', '_')  # Заменяем символы, которые могут быть недопустимыми в имени файла

    # Выполнение GET-запроса для скачивания аудио
    response = requests.get(audio_url)

    # Проверка успешности запроса
    if response.status_code == 200:
        # Сохранение аудио в файл
        with open(file_name, 'wb') as audio_file:
            audio_file.write(response.content)
        print(f"Файл {file_name} успешно скачан.")
    else:
        print(f"Ошибка при скачивании файла: {response.status_code}")

    # Конвертация audio.mp4 в audio.wav
    audio = AudioSegment.from_file(file_name)
    audio.export(f"{lecture_title}.wav", format="wav")  # Сохраняем в формате WAV с названием лекции
    print(f"{lecture_title}.wav успешно сохранен.")


In [ ]:
#@title Скачивание аудио дорожки с VK

def process_vk(url):
    # Параметры для скачивания только аудиодорожки
    ydl_opts = {
        'format': 'bestaudio/best',  # Скачиваем лучшую аудиодорожку
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',  # Используем FFmpeg для извлечения аудио
            'preferredcodec': 'mp3',      # Формат аудио
            'preferredquality': '192',     # Качество аудио
        }],
        'outtmpl': '%(title)s.%(ext)s',  # Шаблон имени выходного файла
    }

    # Скачивание аудиодорожки
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)  # Получаем информацию о видео
        filename = ydl.prepare_filename(info_dict)  # Получаем имя файла
        print(f'Скачанный файл: {filename}')  # Выводим имя файла

        # Получаем метаданные
        lecture_title = info_dict.get('title', None)
        artist = info_dict.get('uploader', None)
        duration = info_dict.get('duration', None)

        print(f'Название: {lecture_title}')
        print(f'Исполнитель: {artist}')
        print(f'Длительность: {duration} секунд')

    # Форматируем имя файла для сохранения
    file_name_mp3 = f"{lecture_title}.mp3"
    # Конвертация audio.mp4 в audio.wav
    audio = AudioSegment.from_file(file_name_mp3)
    audio.export(f"{lecture_title}.wav", format="wav")

    return lecture_title

In [ ]:
#@title Траснкрибация аудиодорожек

def trancribe_audio (lecture_title):
    # Загрузка аудиофайла
    audio_file_path = f"{lecture_title}.wav"
    audio = AudioSegment.from_file(audio_file_path)

    # Определяем длину сегмента (5 минут)
    segment_length = 5 * 60 * 1000  # 5 минут в миллисекундах

    # Разбиваем аудиофайл на сегменты
    segments = []
    for i in tqdm(range(0, len(audio), segment_length), desc=f"Экспорт сегментов {lecture_title}"):
        segment = audio[i:i + segment_length]
        segment_file_path = f"/content/segment_{i // segment_length}.wav"
        segment.export(segment_file_path, format="wav")
        segments.append((segment_file_path, i))  # Сохраняем путь к файлу и начальное время

    # Запускаем распознавание текста для каждого сегмента
    segments_segments = []
    for segment_file, start_time in tqdm(segments, desc="Распознавание текста"):
        segment_segments, info = model.transcribe(segment_file, beam_size=5)

        print(f"Обнаруженный язык '{info.language}' с вероятностью {round(info.language_probability, 2) * 100} %\n\n")

        segment_segments = list(segment_segments)  # Транскрипция будет доступна здесь

        # Вывод с временными отметками
        for segment in segment_segments:
            # Добавляем начальное время к временным меткам
            adjusted_start = segment.start + start_time / 1000  # Преобразуем миллисекунды в секунды
            adjusted_end = segment.end + start_time / 1000
            print(f"[{adjusted_start:.2f}s -> {adjusted_end:.2f}s] {segment.text}")

        segments_segments.extend(segment_segments)

    # Удаляем временные сегменты, если они больше не нужны
    for segment_file, _ in tqdm(segments, desc="Удаление временных файлов"):
        os.remove(segment_file)

    text_segments = list(segments_segments)  # Транскрипция будет доступна здесь.


    # Сохраняем рассшифровку в текстовый файл
    file_txt_path = f'/content/drive/My Drive/MGPU/audio/{lecture_title}_transcript_small.txt'
    with open(file_txt_path, 'w') as file:
        for i in range(len(text_segments)):
            file.write(text_segments[i].text.strip() + '\n')
    print(f'Текстовый файл сохранен', file_txt_path)

    return file_txt_path

## Функции создания базы знаний

In [ ]:
#@title Функция для загрузки документа по сслыке и в текстовом формате
def load_doc_id_text(file_path):
    # Open the document and read it as plain text
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

In [ ]:
#@title Функция возвращает количество токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

In [ ]:
#@title Функция по формированию чанков базы знаний
def split_text(text, max_count):
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)

    # Подсчет токенов для каждого фрагмента
    fragment_token_counts = [num_tokens_from_string(fragment.page_content, "cl100k_base") for fragment in fragments]


    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_count,
        chunk_overlap=0,
        length_function=lambda x: num_tokens_from_string(x, "cl100k_base")
    )

    source_chunks = [
        Doc(page_content=chunk, metadata=fragment.metadata)
        for fragment in fragments
        for chunk in splitter.split_text(fragment.page_content)
    ]

    # Подсчет токенов для каждого source_chunk
    source_chunk_token_counts = [num_tokens_from_string(chunk.page_content, "cl100k_base") for chunk in source_chunks]


    return source_chunks, fragments

In [ ]:
#@title Объединение чанков на группы
def split_into_groups(chunks, group_size=group_chunk):
    for i in range(0, len(chunks), group_size):
        yield chunks[i:i + group_size]

## Модели GPT

In [ ]:
#@title Модель по формированию ответов на основе отрывков лекций
def answer_index(system, topic, search_index, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=7)
    if verbose: print('\n ===========================================: ')
    #message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n =======\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n =======\nОтрывок лекции №{i+1}\n=====' + str(doc.metadata) + '\n================' + str(doc.page_content) + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return message_content, answer  # возвращает ответ

In [ ]:
#@title Модель по группированию ответа по части лекции
def rewrite_chunks(chunk_group):
    combined_text = " ".join([chunk.page_content for _, chunk in chunk_group])
    prompt = f"{system_part}:\n\n{combined_text}"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2000,  # Ограничение по токенам
        temperature=0.1,  # Настройка креативности
    )

    return response.choices[0].message.content

In [ ]:
#@title  Модель для редактирования и объединения
def combine_and_edit(rewritten_texts):
    combined_text = "\n\n".join(rewritten_texts)
    prompt = f"{system_all}:\n\n{combined_text}"

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Или другая модель
        messages=[{"role": "user", "content": prompt}],
        max_tokens=4000,  # Ограничение по токенам
        temperature=0.1,
    )

    return response.choices[0].message.content

## Основной код

In [ ]:
#@title Получение ссылок лекций из файла

# Путь к файлу Excel
video_list_patch = '/content/drive/MyDrive/MGPU/Лекции.xlsx'

# Путь к папке на Google Диске
folder_path = "/content/drive/My Drive/MGPU/audio/"

# Загрузка файла Excel
df_link = pd.read_excel(video_list_patch)

# Проверка загруженных данных
#print(df_link.head())

# Извлечение ссылок из столбца 'Ссылка на просмотр'
# Предполагаем, что ссылки находятся в строках, разделенных символом новой строки
links = df_link['Ссылка на просмотр'].dropna().tolist()  # Удаляем пустые значения и преобразуем в список

# Разделяем ссылки и убираем лишние пробелы
extracted_links = []
for link in links:
    # Разделяем по символу новой строки и убираем лишние пробелы
    extracted_links.extend([l.strip() for l in link.split('\n') if l.strip()])

# Выводим все извлеченные ссылки
for i, link in enumerate(extracted_links, start=1):
    print(f"{i}: {link}")

1: https://dzen.ru/video/watch/66d0b07bcfdd4f4c6c9d8257
2: https://dzen.ru/video/watch/66d0b2966e333d4678131329
3: https://dzen.ru/video/watch/66d0b4597fa01b0436100ad9
4: https://dzen.ru/video/watch/66d0b4aea6fec84560ee890b
5: https://dzen.ru/video/watch/66d0b4fb6e333d4678198c9b
6: https://dzen.ru/video/watch/66d0b57ffc532104b310d2c5
7: https://dzen.ru/video/watch/66d0b6285576c91824e91370
8: https://dzen.ru/video/watch/66d0b6dd48695e33871377fb
9: https://vk.com/video_ext.php?oid=-226281112&id=456239425&hd=1&hash=3198fade2f08e5b6
10: https://vk.com/video_ext.php?oid=-226281112&id=456239426&hd=1&hash=546fc340e5d55c9b
11: https://vk.com/video_ext.php?oid=-226281112&id=456239427&hd=1&hash=05bf354941f8d3f0
12: https://vk.com/video_ext.php?oid=-226281112&id=456239428&hd=1&hash=a5702db93b1e4208
13: https://dzen.ru/video/watch/66d714fad2b0877c5fc0f592
14: https://dzen.ru/video/watch/66d1fc8adba9601de7c4c297
15: https://dzen.ru/video/watch/66d1fd517ed9f31a9f451f30
16: https://dzen.ru/video/watc

In [ ]:
#@title Обработка ссылок

file_list = []

for link in extracted_links:
    if "dzen.ru" in link:
        print(f"Обрабатываю Dzen ссылку: {link}")
        audio_url, lecture_title = await get_url_dzen(link)  # Вызов функции для Dzen

        if audio_url is None:
            print(f"Не удалось получить аудио для лекции: {lecture_title}. Пропускаем...")
            continue  # Пропускаем итерацию, если аудио не найдено

        print(lecture_title)
        print(audio_url)
        get_dzen_audio(lecture_title, audio_url)

        print (f'Идет распознавание лекции {lecture_title}...')
        file_txt_path = trancribe_audio (lecture_title)
        file_list.append(file_txt_path)

    elif "vk.com" in link:
        print(f"Обрабатываю VK ссылку: {link}")
        lecture_title = process_vk(link)  # Вызов функции для VK

        print (f'Идет распознавание лекции {lecture_title}...')
        file_txt_path = trancribe_audio (lecture_title)
        file_list.append(file_txt_path)

    else:
        print(f"Неизвестный тип ссылки: {link}")

In [ ]:
#@title Формируем датафрейм со списком файлов

# Путь к файлу Excel
video_list_patch = '/content/drive/MyDrive/MGPU/audio/Лекции - видео.xlsx'

# Загрузка файла Excel
df = pd.read_excel(video_list_patch)

# Заполнение объединенных ячеек
df['Дисциплина'] = df['Дисциплина'].fillna(method='ffill')
df['Тема'] = df['Тема'].fillna(method='ffill')

# Создание списка для хранения результатов
result_rows = []

# Обработка каждой строки в исходном датафрейме
for index, row in df.iterrows():
    discipline = row['Дисциплина']
    topic = row['Тема']

    # Обработка ссылок и названий файлов
    if isinstance(row['Ссылка на просмотр'], str):
        links = [link.strip() for link in row['Ссылка на просмотр'].split('\n') if link.strip()]
    else:
        links = []

    if isinstance(row['Название файла'], str):
        names = [name.strip() for name in row['Название файла'].split('\n') if name.strip()]
    else:
        names = []

    # Создание новых строк в результирующем списке
    for link, name in zip(links, names):
        result_rows.append({
            'Дисциплина': discipline,
            'Тема': topic,
            'Ссылка на просмотр': link,
            'Название файла': name
        })

# Создание результирующего датафрейма
result_df = pd.DataFrame(result_rows)

# Выводим результирующий датафрейм
result_df

,Дисциплина,Тема,Ссылка на просмотр,Название файла
0,Стратегии развития современного образования,Тема 1: Тенденции и проблемы развития,https://dzen.ru/video/watch/66d0b07bcfdd4f4c6c...,1371I Афанасьев В.В._Стратегии развития соврем...
1,Стратегии развития современного образования,Тема 1: Тенденции и проблемы развития,https://dzen.ru/video/watch/66d0b2966e333d4678...,1372I Афанасьев В.В._Стратегии развития соврем...
2,Стратегии развития современного образования,Тема 2: Государственная политика РФ в сфере ст...,https://dzen.ru/video/watch/66d0b4597fa01b0436...,1373I Афанасьев В.В._Стратегии развития соврем...
3,Стратегии развития современного образования,Тема 2: Государственная политика РФ в сфере ст...,https://dzen.ru/video/watch/66d0b4aea6fec84560...,1374I Афанасьев В.В._Стратегии развития соврем...
4,Стратегии развития современного образования,Тема 3: Стратегические проекты развития соврем...,https://dzen.ru/video/watch/66d0b4fb6e333d4678...,1375I Афанасьев В.В._Стратегии развития соврем...
5,Стратегии развития современного образования,Тема 3: Стратегические проекты развития соврем...,https://dzen.ru/video/watch/66d0b57ffc532104b3...,1376I Афанасьев В.В._Стратегии развития соврем...
6,Стратегии развития современного образования,Тема 3: Стратегические проекты развития соврем...,https://dzen.ru/video/watch/66d0b6285576c91824...,1377I Афанасьев В.В._Стратегии развития соврем...
7,Стратегии развития современного образования,Тема 3: Стратегические проекты развития соврем...,https://dzen.ru/video/watch/66d0b6dd48695e3387...,1378I Афанасьев В.В._Стратегии развития соврем...
8,"Основы педагогики и образования: история, дида...",Тема 1. История российского образования и педа...,https://vk.com/video_ext.php?oid=-226281112&id...,История российского образования_1_transcript_s...
9,"Основы педагогики и образования: история, дида...",Тема 1. История российского образования и педа...,https://vk.com/video_ext.php?oid=-226281112&id...,История российского образования_2_transcript_s...


In [ ]:
#@title Создание базы знаний лекций в одном файле в разметке Markdown

# Инициализация пустой строки для базы знаний
knowledge_base = ""

# Проход по строкам датафрейма
for index, row in result_df.iterrows():
    discipline = row['Дисциплина']
    topic = row['Тема']
    file_name = row['Название файла']

    # Проверка на пустое имя файла
    if not file_name:
        print(f"Пустое имя файла в строке {index}. Пропускаем.")
        continue

    # Очистка имени файла от недопустимых символов
    file_name = re.sub(r'[^\w\-_. ]', '', file_name)
    file_name = os.path.basename(file_name)  # Извлекаем имя файла, если указан путь
    file_link = f"/content/drive/MyDrive/MGPU/audio/{file_name}"

    # Проверка, является ли file_link файлом
    if os.path.isfile(file_link):
        with open(file_link, 'r', encoding='utf-8') as file:
            content = file.read()
    else:
        print(f"Файл {file_name} не найден или не является файлом.")
        content = "Файл отсутствует."

    # Формирование текста
    knowledge_base += f"# {discipline}\n"
    knowledge_base += f"## {topic}\n"
    knowledge_base += f"### {file_name}\n"
    knowledge_base += f"{content}\n\n"

# Сохранение базы знаний в текстовый файл
with open('knowledge_base.txt', 'w', encoding='utf-8') as f:
    f.write(knowledge_base)

print("База знаний успешно сформирована и сохранена в 'knowledge_base.txt'.")

NameError: name 'result_df' is not defined

In [ ]:
#@title Загрузка базы знаний из текстового формата
data_from_markdown = load_doc_id_text('/content/knowledge_base.txt')
#Проверка базы
print(data_from_markdown[-100:])
print(len(data_from_markdown))

In [ ]:
#@title Получаем чанки базы знаний
source_chunks, fragments = split_text(data_from_markdown, pcs_token)
#Проверка чанков
print("Общее количество чанков: ",len(source_chunks))
print("\n\nПервый чанк ", source_chunks[0])
print("\n\nMetadata Первого чанка", source_chunks[0].metadata)
print("\n\nКрайний чанк ", source_chunks[len(source_chunks)-1])
print("\n\nMetadata крайнего чанка", source_chunks[len(source_chunks)-1].metadata)

In [ ]:
#@title Создание и сохранение индексной базы (embedding)
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

# Сохраним индекс в файл
db.save_local(f"{folder_path}faiss_index.bin")

NameError: name 'OpenAIEmbeddings' is not defined

In [ ]:
#@title Ответ на вопрос пользователя
topic= "Новое технологическое образование в школе и среднем профессиональном образовании – необходимое условие развития российского образования."
message, ans = answer_index(system, topic, db)
print(message)
print(f'\n =======\nОтвет GPT:\n{ans}')


Отрывок лекции №1
====={'Header 1': 'Стратегии развития современного образования', 'Header 2': 'Тема 3: Стратегические проекты развития современного российского образования.', 'Header 3': '1378I Афанасьев В.В._Стратегии развития современного образования_2.4_transcript_small.txt'}
================Пятое, новые технологические образования в школе и в структуре среднепрессионального
образования, проблема тоже довольно-таки серьезная, что программы не меняются и в школе,
вот в этих образовательных организациях незначительное количество детей в
частых в программах технологической направленности, я уже тоже об этом
рассуждал, то есть, Московская региона это одно, Россия это другое, то есть мы сейчас напоминаем вам
говорим о российской действительности. Технологические компетенции, ну они не соответствуют,
конечно, требованиям рынка труда, даже вот я сейчас вспоминаю, очень мало научно даже
работ, которые развивают, формируют, создают условия, педагогические условия для того,
чтобы эти технол

In [ ]:
#@title Ответ на вопрос пользователя
topic= "Новое технологическое образование в школе и среднем профессиональном образовании – необходимое условие развития российского образования."
message, ans = answer_index(system, topic, db)
print(message)
print(f'\n =======\nОтвет GPT:\n{ans}')


Отрывок лекции №1
====={'Header 1': 'Стратегии развития современного образования', 'Header 2': 'Тема 3: Стратегические проекты развития современного российского образования.', 'Header 3': '1378I Афанасьев В.В._Стратегии развития современного образования_2.4_transcript_small.txt'}
================Пятое, новые технологические образования в школе и в структуре среднепрессионального
образования, проблема тоже довольно-таки серьезная, что программы не меняются и в школе,
вот в этих образовательных организациях незначительное количество детей в
частых в программах технологической направленности, я уже тоже об этом
рассуждал, то есть, Московская региона это одно, Россия это другое, то есть мы сейчас напоминаем вам
говорим о российской действительности. Технологические компетенции, ну они не соответствуют,
конечно, требованиям рынка труда, даже вот я сейчас вспоминаю, очень мало научно даже
работ, которые развивают, формируют, создают условия, педагогические условия для того,
чтобы эти технол

In [ ]:
#@title Ответ на вопрос пользователя
topic= "Закономерности и особенности психического развития на этапе подросткового возраста и проблемы психологии образования"
message, ans = answer_index(system, topic, db)
print(message)
print(f'\n =======\nОтвет GPT:\n{ans}')


Отрывок лекции №1
====={'Header 1': 'Психология образования', 'Header 2': 'Тема 2: Психологические закономерности и механизмы познавательного и личностного развития ребенка на первых этапах онтогенеза', 'Header 3': '1465I Тема 3. Ч.1._Психологические закономерности и механизмы_transcript_small.txt'}
================Здравствуйте, уважаемые студенты! Тема психологической
закономерности и механизм и познавательного илического
развития ребенка на первых этапах от тегенеза рассматривается
нами в рамках одного из ключевых направлений психология образования,
возрастная психология образования. В этом фрагменте лекции мы рассмотрим
закономерности и механизмы психического развития. В этой виде лекции мы рассмотрим такие вопросы,
как возрастные периоды развития, причины и механизмы перехода этого возрастного
периода к другому. Общие закономерности и тенденции, темп и направленность психического
развития в антагенезе. Рассмотрим также психические особенности возрастных стадий,
вошедших в периодиз

In [ ]:
#@title Пересказ лекции

In [ ]:
#@title Загрузка индексной базы из файла
# Загрузка индекса с разрешением на десериализацию
db_loaded = FAISS.load_local(
    folder_path=index_path,
    embeddings=OpenAIEmbeddings(),  # Убедитесь, что embeddings инициализированы правильно
    allow_dangerous_deserialization=True  # Разрешение на загрузку
)

print("Индекс успешно загружен!")

Индекс успешно загружен!


In [ ]:
#@title Получение списка лекций
# Получение всех идентификаторов документов
doc_ids = db_loaded.index_to_docstore_id.values()

# Получение метаданных для каждого чанка
chunks_with_metadata = [db_loaded.docstore.search(doc_id) for doc_id in doc_ids]

print ('Всего чанков:', len (chunks_with_metadata))

# Получаем уникальные значения из столбца 'Название файла'
unique_files = result_df['Название файла'].unique()

# Если нужно преобразовать результат в список
unique_files_list = unique_files.tolist()

print(f'Список лекций:\n{unique_files}')


Всего чанков: 173
Список лекций:
['1371I Афанасьев В.В._Стратегии развития современного образования_1.1_transcript_small.txt'
 '1372I Афанасьев В.В._Стратегии развития современного образования_1.2_transcript_small.txt'
 '1373I Афанасьев В.В._Стратегии развития современного образования_1.3_transcript_small.txt'
 '1374I Афанасьев В.В._Стратегии развития современного образования_1.4_transcript_small.txt'
 '1375I Афанасьев В.В._Стратегии развития современного образования_2.1_transcript_small.txt'
 '1376I Афанасьев В.В._Стратегии развития современного образования_2.2_transcript_small.txt'
 '1377I Афанасьев В.В._Стратегии развития современного образования_2.3_transcript_small.txt'
 '1378I Афанасьев В.В._Стратегии развития современного образования_2.4_transcript_small.txt'
 'История российского образования_1_transcript_small.txt'
 'История российского образования_2_transcript_small.txt'
 'История российского образования_3_transcript_small.txt'
 'История российского образования_4_transcript_sm

In [ ]:
#@title Настройки пересказа лекции: Выберите лекцию и количество чанков для подачи в модель
#@title Выбор названия лекции
target_header_3 = 'ID1582F Фролова Е.В._Лекция 4_Основы когнитивной психологии_Осн_когн_психол_Л4_transcript_small.txt' #@param ['1371I Афанасьев В.В._Стратегии развития современного образования_1.1_transcript_small.txt', '1372I Афанасьев В.В._Стратегии развития современного образования_1.2_transcript_small.txt', '1373I Афанасьев В.В._Стратегии развития современного образования_1.3_transcript_small.txt', '1374I Афанасьев В.В._Стратегии развития современного образования_1.4_transcript_small.txt', '1375I Афанасьев В.В._Стратегии развития современного образования_2.1_transcript_small.txt', '1376I Афанасьев В.В._Стратегии развития современного образования_2.2_transcript_small.txt', '1377I Афанасьев В.В._Стратегии развития современного образования_2.3_transcript_small.txt', '1378I Афанасьев В.В._Стратегии развития современного образования_2.4_transcript_small.txt', 'История российского образования_1_transcript_small.txt', 'История российского образования_2_transcript_small.txt', 'История российского образования_3_transcript_small.txt', 'История российского образования_4_transcript_small.txt', 'ID1587 Никтина Э.К._Основы педагогики и образования история, дидактика, теория воспитания_Дидактика_3_transcript_small.txt', '1459I Тема 2. Ч. 1._Базовые теории учения_transcript_small.txt', '1460I Тема 2. Ч. 2._Базовые теории учения__transcript_small.txt', '1465I Тема 3. Ч.1._Психологические закономерности и механизмы_transcript_small.txt', '1472I Тема 5. Ч.1._Социальная пс. образования_transcript_small.txt', '1473I Тема 5. Ч.2._Социальная пс. образования_transcript_small.txt', '1474I Тема 8. Ч1._Психол. служба в образовании_transcript_small.txt', '1475I Тема 8. Ч2._Психол. служба в образовании_transcript_small.txt', 'Лекция 1_Методология и методы научного исследования_Степанов СЮ_transcript_small.txt', '1515I Лекция 2_Методология и методы научного исследования_Степанов СЮ_transcript_small.txt', '1498I Лекция 4-1_Методология_Суннатова_магистры_transcript_small.txt', '1499I Лекция 4-2_Методология_Суннатова_магистры_transcript_small.txt', '1500I Лекция 5-1_Методология_Суннатова_магистры_transcript_small.txt', '1501I Лекция 5-2_Методология_Суннатова_магистры_transcript_small.txt', '1502I Лекция 5-3_Методология_Суннатова_магистры_transcript_small.txt', 'ID1569F Фролова Е.В._Лекция 1_Основы когнитивной психологии_Осн_когн_психол_Л1_transcript_small.txt', 'ID1583F Фролова Е.В._Лекция 2_Основы когнитивной психологии_Осн_когн_психол_Л2_transcript_small.txt', 'ID1576F Фролова Е.В._Лекция 3_Основы когнитивной психологии_Осн_когн_психол_Л3_transcript_small.txt', 'ID1582F Фролова Е.В._Лекция 4_Основы когнитивной психологии_Осн_когн_психол_Л4_transcript_small.txt', '1439I Поставнев В.М._Психология индивид различий_Тема 2. Ч. 1. Предпосылки и условия индивидуальных различий__transcript_small.txt', '1440I Поставнев В.М._Психология индивид различий_Тема 2. Ч. 2. Предпосылки и условия индивидуальных различий__transcript_small.txt', '1441I Поставнев В.М._Психология индивид различий_Тема 3. Ч.1. Темперамент__transcript_small.txt', '1442I Поставнев В.М._Психология индивид различий_Тема 3. Ч.2 Темперамент]_transcript_small.txt', 'ID336 Листик Психология индивидуальных различий как отрасль психологии_transcript_small.txt', '1496I Лекция 1_Часть 2_Листик_Введение в психологию индивидуальных различий_transcript_small.txt', 'ID1532Листик Е.М._Тема 2. Интеллект как центральная проблема психологии индивидуальных различий. Способности и интеллектЕ_transcript_small.txt']
#Количество чанков объединяющихся для перессказа
group_chunk = 4 #@param {type: 'integer'}


In [ ]:
#@title Рерайтинг объединенных чанков лекций

# Фильтрация чанков, соответствующих конкретному Header 3 (лекции)
filtered_chunks = [
    (i, chunk) for i, chunk in enumerate(chunks_with_metadata)
    if chunk.metadata.get("Header 3") == target_header_3
]

'''
# Вывод индексов и содержимого чанков
for i, chunk in filtered_chunks:
    print(f"Индекс чанка: {i}")
    print(f"Заголовок (Header 3): {chunk.metadata['Header 3']}")
    print(f"Содержимое: {chunk.page_content}")
    print("-" * 50)
'''
#Количество чанков
#print (len (filtered_chunks))

chunk_groups = list(split_into_groups(filtered_chunks, group_size=group_chunk))
#print(f"Количество объединенных групп для рерайтинга: {len(chunk_groups)}")

rewritten_texts = []
for group in chunk_groups:
    time.sleep(1)
    rewritten_text = rewrite_chunks(group)
    rewritten_texts.append(rewritten_text)
    print(f"Отрерайченный текст:\n{rewritten_text}\n")

Отрерайченный текст:
Здравствуйте, уважаемые студенты. 

Сегодня мы обсудим интеллект как ключевую проблему в области психологии индивидуальных различий. Меня зовут Листика Елена Мариковна, я являюсь доцентом Департамента Психологии в Институте Педагогики и Психологии образования.

На этом слайде представлен план нашей лекции. Мы рассмотрим несколько ключевых понятий. Исследование интеллекта занимает важное место в дифференциальной психологии, и начнем мы с определения самого интеллекта. Слово "интеллект" происходит от латинского и переводится как "понимание" или "познание". В настоящее время в психологической литературе нет единого мнения о том, как именно следует определять интеллект. Более того, при попытке его определить необходимо учитывать как минимум три его разновидности: биологический, социальный и психометрический.

Биологический интеллект связан с функциями коры головного мозга. По мнению английского психолога Ганса Юргена Айзенка, он является физиологической, биохимической 

In [ ]:
#@title  Модель для редактирования и объединения для пересказа Лекции
def combine_and_edit(rewritten_texts):
    combined_text = "\n\n".join(rewritten_texts)
    prompt = f"Объедини текст в единый пересказ, устрани повторы и улучши связность:\n\n{combined_text}"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=4000,
        temperature=0.1,
    )

    return response.choices[0].message.content

# объединение и редактирование
final_summary = combine_and_edit(rewritten_texts)
print(f"Итоговый пересказ лекции:\n{final_summary}")

Итоговый пересказ лекции:
Здравствуйте, уважаемые студенты!

Сегодня мы обсудим интеллект как ключевую проблему в области психологии индивидуальных различий. Меня зовут Листика Елена Мариковна, я доцент Департамента Психологии в Институте Педагогики и Психологии образования. В ходе лекции мы рассмотрим основные понятия, связанные с интеллектом, его определение и различные подходы к его изучению.

Слово "интеллект" происходит от латинского и переводится как "понимание" или "познание". В психологической литературе нет единого мнения о том, как именно следует определять интеллект, однако можно выделить три его разновидности: биологический, социальный и психометрический. Биологический интеллект связан с функциями коры головного мозга и зависит от генетических факторов. Социальный интеллект формируется в результате взаимодействия с окружающей средой и проявляется в социальной адаптации. Психометрический интеллект представляет собой статистическое понятие, измеряемое с помощью тестов.

Канад